# Upper Green PEST Pilot Point Setup

In [ ]:
%matplotlib inline
import os, shutil
import sys
sys.path.append("..")
import numpy as np
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt

import flopy as flopy
import pyemu
import shapefile #the pyshp module
from pyemu.pst.pst_utils import SFMT,IFMT,FFMT,pst_config

# <span style="color:yellow">1. Set up pilot points network for Green model some</span>.

There are multiple approaches to implementing pilot points with PEST++.  

In this class, we will use some kick-ass pyemu sweetness

### 1.1. Set up zones for where pilot points will be interpolated

We can have pilot point networks in multiple zones. In this case, we will make a simple zone file using `IBOUND` such that all active cells are in the same interpolation zone.

In [ ]:
working_dir = 'D:/Projects/Watersheds/Green/Analysis/APEX-MODFLOWs/gr_012721/APEX-MODFLOW/MODFLOW'
mname = "gr_1000.nam"
os.chdir(working_dir)




In [ ]:
# m = flopy.modflow.Modflow.load(fs.MODEL_NAM,model_ws=wd,load_only=[]) #<-- load only prevents reading ibound
m = flopy.modflow.Modflow.load(
            mname,
            model_ws=working_dir
            )
m.check()

In [ ]:
m.bas6.ibound[0].plot()

### 1.2. It is for when pilot points don't exist. We don't want pilot points or care about HK values in inactive cells, but we do need values in constant heads

We are going to use a pyemu helper function to setup pilot points are cell centers for active cells.

In [ ]:
# Create pilot points as a shapefile
# we want hk pilot points in the top layer...
prefix_dict = {0:["sy0"]}
df_pp_hk = pyemu.pp_utils.setup_pilotpoints_grid(ml=m,
                                              prefix_dict=prefix_dict,
                                              pp_dir=working_dir,
                                              tpl_dir=working_dir,
                                              every_n_cell=10,
                                              shapename='pp_sy.shp')
# pp_file = os.path.join(working_dir,"sypp.dat")
# assert os.path.exists(pp_file)


### 1.3. Create dataframe from shapefile

#### **1.3.1. Change Shapefile name**

In [ ]:
# change shapefile and file name
shpwd = working_dir
shp = 'pp_sy.shp'
shp_changed = 'sy0pp.shp'
ppf = shp_changed[:-3] + 'dat'
ppf

#### **1.3.2. Shapefile to Dataframe**

In [ ]:
#read file, parse out the records and shapes
shapefile_path = os.path.join(shpwd, shp)
sf = shapefile.Reader(shapefile_path)

#grab the shapefile's field names (omit the first psuedo field)
fields = [x[0] for x in sf.fields][1:]
records = sf.records()
shps = [s.points for s in sf.shapes()]

#write the records into a dataframe
shapefile_dataframe = pd.DataFrame(columns=fields, data=records)

#add the coordinate data to a column called "coords"
shapefile_dataframe = shapefile_dataframe.assign(coords=shps)

pp_df = shapefile_dataframe.sort_values(by=['name'])
print(pp_df)

In [ ]:
os.chdir(working_dir)

In [ ]:
os.getcwd()

In [ ]:
pyemu.utils.pp_utils.write_pp_file(ppf, pp_df)
# pyemu.utils.pp_utils.pilot_points_to_tpl('hahaha.dat')

In [ ]:
PP_FMT = {"name": SFMT, "x": FFMT, "y": FFMT, "zone": IFMT, "tpl": SFMT,
          "parval1": FFMT}
def pp_to_tpl(pp_file, tpl_file=None):
    names = pp_df['parnme'].tolist() # for hk
    # names = pp_df['sypar'].tolist() # for sy  
#     names = pp_df['parnme'].tolist() # for river conductance      
    if tpl_file is None:
        tpl_file = pp_file + ".tpl"    
    tpl_entries = ["~    {0}    ~".format(name) for name in names]
    pp_df.loc[:,"tpl"] = tpl_entries
    pp_df.loc[:,"parnme"] = names


    f_tpl = open(tpl_file,'w')
    f_tpl.write("ptf ~\n")
    f_tpl.write(pp_df.to_string(col_space=0,
                              columns=["name","x","y","zone","tpl"],
                              formatters=PP_FMT,
                              justify="left",
                              header=False,
                              index=False) + '\n')    
pp_to_tpl(ppf)

Let's look at ``pp_df`` - it has a lot of useful info

In [ ]:
pp_df

So cool, we now defined pilot points as a set of spatially distributed parameters...but how do go from pilot points to the model input HK array? Answer: geostatistics.  We need to calculate the geostatistical factors (weights) used to form the interpolated value for the HK value at each model cell - its a spatially-weighted combination of pilot point values

# <span style="color:blue">2. Geostatistics</span>

## Need to create Kriging factors and regularization inputs
Following the guidelines in _Approaches to Highly Parameterized Inversion: Pilot-Point Theory, Guidelines, and Research Directions_ https://pubs.usgs.gov/sir/2010/5168/

### First we need to define a couple geostatistical structures (e.g. variograms)

From _PEST Groundwater Data Utilities Part A: Overview_ page 43, there are 4 acceptable variogram types:

 1. *Spherical*  
### $\gamma\left(h\right)=c\times\left[1.5\frac{h}{a}-0.5\frac{h}{a}^3\right]$ if $h<a$
### $\gamma\left(h\right)=c$ if $h \ge a$  
     
 2. *Exponential*  
### $\gamma\left(h\right)=c\times\left[1-\exp\left(-\frac{h}{a}\right)\right]$  
     
 3. *Gaussian*  
### $\gamma\left(h\right)=c\times\left[1-\exp\left(-\frac{h^2}{a^2}\right)\right]$  
 
 4. *Power*  
### $\gamma\left(h\right)=c\times h^a$
     
 The number refers to `VARTYPE`. `BEARING` and `ANISOTROPY` only apply if there is a principal direction of anisotropy. $h$ is the separation distance, and $a$ is the range, expressed with the `A` parameter.


### First, let's create ``variogram`` and ``GeoStruct`` objects.  

These describe how HK varies spatailly, remember?

In [ ]:
v = pyemu.geostats.ExpVario(contribution=0.8,a=30000, bearing=0)
gs = pyemu.geostats.GeoStruct(variograms=v,nugget=0.0)
ax = gs.plot()
ax.grid()
# ax.set_ylim(0,2.0)

Now, let's get an ``OrdinaryKrige`` object, which needs the ``GeoStruct`` as well as the x, y, and name of the pilot point locations (which happens to be in that really cool ``df_pp`` instance)

In [ ]:
ok = pyemu.geostats.OrdinaryKrige(gs,pp_df)

Once the ``OrdinaryKrige`` is created, we need to calculate the geostatistical interpolation factors for each model cell.  We do this with the ``.calc_factors_grid()`` method: it needs to know about the model's spatial orientation and also accepts some optional arguments:

### Kriging Processing... it takes time

In [ ]:
df = ok.calc_factors_grid(m.sr,
#                           var_filename=pst_name.replace(".pst",".var.ref"),
                          var_filename= ppf[:-3] + "var.ref",                          
                          minpts_interp=1,maxpts_interp=50,
                          search_radius=1000000000000.0)


One of the really cool things about geostatistics is that it gives you both the interpolation (factors), but also gives you the uncertainty in the areas between control (pilot) points.  Above, we wrote this uncertainty information to an array that has the same rows and cols as the model grid - this array is very useful for understanding the function of the variogram.

In [ ]:
# arr_var = np.loadtxt(pst_name.replace(".pst",".var.ref"))
arr_var = np.loadtxt(ppf[:-3] + "var.ref")
ax = plt.subplot(111,aspect="equal")
p = ax.imshow(arr_var,extent=m.sr.get_extent(),alpha=0.25)
plt.colorbar(p)
plt.tight_layout()
ax.scatter(pp_df.x,pp_df.y,marker='.',s=4,color='r')

We see that at the pilot point locations (red dots), the uncertainty in the geostats is minimal...as expected. The call to ``.calc_factors_grid()`` also returns a ``DataFrame`` which has useful info - lets look:

In [ ]:
df

We see that there is one row for each model cell, and for each row, we see the distance, names, and weight for the "nearby" pilot points.  The interpolated value for cells that have a pilot point at their center only need one weight - 1.0 - and one pilot point.  Other cells are weighted combinations of pilot points.  Is this clear?  

### Now we need to save the factors (weights) to a special file that we will use later to quickly generate a new HK array from a set of pilot point values:

In [ ]:
ok.to_grid_factors_file(ppf+".fac")

Just for demo purposes, lets generate ``random`` pilot point values and run them through the factors to see what the ``hk`` array looks like

In [ ]:
# generate random values
pp_df.loc[:,"parval1"] = np.random.random(pp_df.shape[0])
# save a pilot points file
pyemu.pp_utils.write_pp_file(ppf,pp_df)

In [ ]:
# interpolate the pilot point values to the grid
hk_arr = pyemu.utils.geostats.fac2real(ppf,factors_file=ppf+".fac",out_file=None)

In [ ]:
# plot
ax = plt.subplot(111,aspect='equal')
ax.imshow(hk_arr,interpolation="nearest",extent=m.sr.get_extent(),alpha=0.5)
ax.scatter(pp_df.x,pp_df.y,marker='.',s=4,color='k')

What happens if you recalculate the factors using one point for every cell? Change ``max_interp_pts`` to 1 in the ``calc_factors_grid()`` and rerun these cells...

### An aside on geostatistics and covariance matrices

The ``GeoStruct`` object above was used to interpolate from pilot point locations to each node in the grid.  But this same ``GoeStruct`` also has an important information regarding how the pilot points are related to each other spatially---that is, the ``GeoStruct`` object implies a covariance matrix.  Let's form that matrix 

In [ ]:
cov = gs.covariance_matrix(pp_df.x,pp_df.y,pp_df.parnme)

In [ ]:
plt.imshow(cov.x)

In [ ]:
cov.to_dataframe()

What do these numbers mean?  Why should you care?  Well, this covariance matrix plays an important role in uncertainty quantification, as well as in governing the way pilot point parameters are adjusted during calibration

# Build instruction files (Streamflow / Watertable)

### 1. Streamflow (output.rch)

In [ ]:
sys.path.insert(0, 'D:/spark-brc_gits/apexmf_git/apexmf_pkgs')
# from apexmf_pst_pkgs import apexmf_pst_utils, apexmf_pst_par
import apexmf_pst_utils

In [ ]:
working_dir = "D:/Projects/Watersheds/Green/Analysis/APEX-MODFLOWs/calibrations/gr_210614/APEX-MODFLOW"
os.chdir(working_dir)
wd = "D:/Projects/Watersheds/Green/Analysis/APEX-MODFLOWs/calibrations/gr_210614/APEX-MODFLOW/MODFLOW"



In [ ]:
# Create parm template file

sw_par = apexmf_pst_utils.parm_to_tpl_file()
sw_par

### 1.1.1 Create river parameters


In [ ]:
# provide channel ids that will be used for calibration
subs = ['rg009', 'rg096', 'rg199', 'rg155', 'rg200']
apexmf_pst_par.create_riv_par(wd, subs)

In [ ]:
# create a template file for mf_riv.par file
apexmf_pst_utils.riv_par_to_template_file('mf_riv.par')

In [ ]:
# overwrite the river package file
apexmf_pst_par.riv_par(wd)

# 1.2. Build instruction files (streamflow / watertable / baseflow)
## 1.2.1. Streamflow (output.rch)

In [ ]:
os.chdir(working_dir)

In [ ]:
# file path
rch_file = 'SITE199.RCH'
# reach numbers that are used for calibration
subs = [9, 96, 199]
# extract month_streamflow
apexmf_pst_utils.extract_month_str(rch_file, subs, '1/1/1990', '1/1/2000', '12/31/2012')

### 1.2.3. Create instruction files for each str_sim file using the 'streamflow.obd' file

In [ ]:
# because we have 3 streamgages let's loop for them
# read streamobd and get column names
stf_obd = pd.read_csv(
                    'streamflow_month.obd',
                    sep='\t',
                    index_col=0,
                    parse_dates=True,
                    na_values=[-999, '']
                    )
# stf_obd_c = stf_obd.resample('M').mean()
# stf_obd_c.to_csv('streamflow_m.obd', sep='\t', na_rep=-999, float_format='%.2f')
obds = stf_obd.columns.tolist()[::-1]
# obds.remove('sub046')
# obds.remove('sub130')
print(obds)
sim_files = ['cha_{:03d}.txt'.format(x) for x in subs]
# sed_files = ['sed_{:03d}.txt'.format(x) for x in subs]
# sim_files = sim_files + sed_files
print(sim_files)

In [ ]:
# create instruction files for each sim file
for i in range(len(sim_files)):
    apexmf_pst_utils.stf_obd_to_ins(sim_files[i], obds[i], '1/1/2000', '12/31/2012', time_step='month')

# Get groundwater levels

In [ ]:
wtdf = pd.read_excel("D:/Projects/Watersheds/Green/GIS/mf_obs_grids.dbf.xlsx", engine="openpyxl")
# get cal only
wtdf = wtdf.loc[wtdf.cal_counts > 0]
wtdf

In [ ]:
# os.chdir(wd)
os.getcwd()

In [ ]:
# We do have watertable data now
grid_ids = wtdf.grid_id.to_list()

apexmf_pst_utils.extract_watertable_sim(grid_ids, '1/1/1995', '12/31/2012')

In [ ]:
len(grid_ids)

In [ ]:
obd_names = wtdf.name.to_list()

In [ ]:
'wt{:05d}'.format(3087)


In [ ]:
for grid_id, obd_name in zip(grid_ids, obd_names):
    apexmf_pst_utils.mf_obd_to_ins('wt_{}.txt'.format(grid_id), 'wt{:05d}'.format(grid_id), '1/1/1995', '12/31/2012')

In [ ]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'green_dummy.pst')

# print(os.chdir(".."))
io_files

The ``parse_dir_for_io_files()`` helper is looking for files with the ".tpl" and ".ins" extension.  This assumes that the corresponding model input and model output files are the same name, minus the ".tpl" and ".ins" extension, respectively.  These file lists are then passed to another helper, which builds a basic control file for you (``Pst.from_io_files()``).  Let's look at this generic ``Pst`` instance:

In [ ]:
par = pst.parameter_data
par

## 2.1. Change parameter group name

In [ ]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'sy':
        par.iloc[i, 6] = 'sy'
    elif par.iloc[i, 0][:7] == 'rivbot_':
        par.iloc[i, 6] = 'rivbot'
    elif par.iloc[i, 0][:6] == 'rivcd_':
        par.iloc[i, 6] = 'rivcd'
    elif par.iloc[i, 0][:2] == 'hk':
        par.iloc[i, 6] = 'hk'
    elif par.iloc[i, 0][:1] == 'p':
        par.iloc[i, 6] = 'apex'
    # else:
    #     par.iloc[i, 6] = 'str'
par

In [ ]:
par = par.sort_values(by=['pargp', 'parnme'])
par

### 3. Set par ranges and values for MODFLOW

In [ ]:
# for MODFLOW parameters
for i in range(len(par)):
    if (par.iloc[i, 6]) == 'hk':
        par.iloc[i, 4] = 1.100000e-03
        par.iloc[i, 5] = 1.100000e+03
    elif par.iloc[i, 6] == 'sy':
        par.iloc[i, 3] = 0.100000e+00 
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 0.800000e+00        
    elif par.iloc[i, 6] == 'rivcd':
        par.iloc[i, 3] = 50.01   # initial    
        par.iloc[i, 4] = 0.1   # lower
        par.iloc[i, 5] = 100   # upper
        par.iloc[i, 8] = -50   # offset
    elif par.iloc[i, 6] == 'rivbot':
        par.iloc[i, 3] = 5.0001   # initial    
        par.iloc[i, 4] = 0.1   # lower
        par.iloc[i, 5] = 10   # upper
        par.iloc[i, 8] = -5   # offset
#     else:
#         par.iloc[i, 6] = 'str_par'


In [ ]:
par

# APEX

In [ ]:
pst.parameter_data = apexmf_pst_utils.export_pardb_pest(par)

In [ ]:
par = pst.parameter_data
par


Cool - the other tpl files were found and parsed - parameter listed in them were added to the control file.  But we have generic entries for initial values bounds...

## Observation

In [ ]:
obd = pst.observation_data
print(obd)

In [ ]:
for i in range(len(obd)):
    if obd.iloc[i, 0][:6] == 'sub009':
        obd.iloc[i, 3] = 'sub009'
    elif obd.iloc[i, 0][:6] == 'sub096':
        obd.iloc[i, 3] = 'sub096'
    elif obd.iloc[i, 0][:6] == 'sub199':
        obd.iloc[i, 3] = 'sub199'
    else:
        obd.iloc[i, 3] = obd.iloc[i, 0][:7]


In [ ]:
print(obd)

## 2.3. Import measured data

In [ ]:
gwt_obd = pd.read_csv('MODFLOW/modflow.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
gwt_obd = gwt_obd['1/1/2000': '12/31/2012']
gwt_obd = gwt_obd.dropna(axis=1, how='all')

gwtcolnams = gwt_obd.columns.tolist()
# gwtcols = [i if len(i) < 7 for i in gwtcolnams]
gwtcols = [i for i in gwtcolnams if len(i) <= 7]



In [ ]:
gwt_obd = gwt_obd[gwtcols]
gwt_obd = gwt_obd.reindex(sorted(gwt_obd.columns), axis=1)
gwt_obd

In [ ]:
gwtcols

In [ ]:
stf_obd = pd.read_csv('streamflow_month.obd',
                       sep='\t',
                       index_col = 0,
                       parse_dates = True,
                       na_values=[-999, '']
                     )
stf_obd = stf_obd['1/1/2000': '12/31/2012']
# stf_obd = stf_obd.drop(['sub046', 'sub130'], axis=1)
stf_obd =  stf_obd.reindex(sorted(stf_obd.columns), axis=1)
stf_obd

In [ ]:
# Get sub list based on obd order
sub_order = []
for i in obd.obgnme.tolist():
    if i not in sub_order:
        sub_order.append(i)
sub_order


In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
for i in sub_order[:3]:
    tot_obd += stf_obd[i].dropna().tolist()
for j in sub_order[3:]:
    tot_obd += gwt_obd[j].dropna().tolist()    
len(tot_obd)
# tot_obd

In [ ]:
obd.loc[:, 'obsval'] = tot_obd
obd

### 4. Export control file

In [ ]:
pst.control_data.noptmax=0
pst.model_command = 'python forward_run.py'
pst.write('green_pest.pst')

also cool - the instruction files in the directory were also found and parsed so that observation listed in the instruction files were added as well. There are some subtlies here, but we will skip them for now.

## Regularization

Regularization is ....

in pyemu, we can add two forms of regularization:
- preferred value: we want the parameter values to stay as close to the initial values as possible
- preferred difference: we prefer the differences in parameter values to be minimized

Preferred value is easy to understand, we simply add ``prior_information`` to the control file to enforce this condition.  pyemu uses a helper for this:

In [ ]:
# load the pre-constructed pst
pst = pyemu.Pst(os.path.join(working_dir,pst_name))

In [ ]:
pyemu.helpers.zero_order_tikhonov(pst,parbounds=False)

In [ ]:
pst.prior_information

Ok, that's fine, but should the weight on preferring HK not to change be the same as preferring recharge not to change?  Seems like we would want recharge to change less than HK.  This preference can be expressed by using the parameter bounds to form the weights

In [ ]:
pyemu.helpers.zero_order_tikhonov(pst,parbounds=True)

In [ ]:
pst.prior_information

Now we are really preferring recharge not to change...good!

So what about preferred difference regularization?  Well pyemu can do that too.  Remember that ``Cov``ariance matrix we built above? It expresses the spatial relationship between pilot points, so we use to setup these prior information equations:

In [ ]:
pyemu.helpers.first_order_pearson_tikhonov(pst,cov)

In [ ]:
pst.prior_information

What happened?  We replace the preferred value equations with a bunch of new equations.  These equations each include two parameter names and have different weights - can you guess what the weights are?  The weights are the pearson correlation coefficients (CC) between the pilot points (remember those from way back?).  These CC values are calculated from the covariance matrix, which is implied by the geostatistical structure...whew! 

# For river Bed conductance interpolation

In [ ]:
# interpolate the pilot point values to the grid
riv_cond = pyemu.gw_utils.fac2real(ppf,factors_file=ppf+".fac",out_file=None)

In [ ]:
np.shape(riv_cond)

## 1. get only river grids

In [ ]:
df_riv = pd.read_csv(
                    shpwd + "\\mf\\ss_072519.riv",
                    delim_whitespace=True,
                    skiprows=3,
#                     usecols=[1,2],
                    header=None
                    )

In [ ]:
new_riv_cf = [riv_cond[df_riv.iloc[i, 0], df_riv.iloc[i, 1]] for i in range(len(df_riv))]

In [ ]:
df_riv.iloc[:, 4] = new_riv_cf
df_riv.iloc[:, 4] = df_riv.iloc[:, 4].map(lambda x: '{:.10e}'.format(x))
df_riv.iloc[:, 3] = df_riv.iloc[:, 3].map(lambda x: '{:.10e}'.format(x))
df_riv.iloc[:, 5] = df_riv.iloc[:, 5].map(lambda x: '{:.10e}'.format(x))

In [ ]:
with open(os.path.join(shpwd + "\\mf", "ss_072519.riv"), 'w') as f:
    f.write("# RIV: River package file created on 7/25/2019 by ModelMuse version 4.0.0.0." + "\n")
    f.write("  1467     9 AUXILIARY IFACE # DataSet 2: MXACTC IRIVCB Option" + "\n")
    f.write("  1467     0 # Data Set 5: ITMP NP Stress period 1" + "\n")    
    df_riv.to_csv(f, sep='\t',
                  header=False,
                  index=False,
#                   float_format='%.2f', 
                  line_terminator='\n', 
                  encoding='utf-8')


# Build instruction files (Streamflow / Watertable)

### 1. Streamflow (channel_day.txt)

In [ ]:
import csv

wd2 = "D:\\Projects\\MiddleBosque\\Analysis\\SWAT+MODFLOW Model_middle_bosque"
df_str = pd.read_csv(
                    wd2 + "\\channel_day - Copy.txt",
                    delim_whitespace=True,
                    skiprows=3,
#                     usecols=[1,2],
                    header=None
                    )
test = []
for i in range(len(df_str)):
    if df_str.iloc[i, 6] == 'cha53':
        a = 'l1 w w w w w w w w w '
        b = '!str_{}{:02d}{:02d}!'.format(df_str.iloc[i, 3], df_str.iloc[i, 1], df_str.iloc[i, 2])
        test.append(a+b)
    else:
        a = 'l1'
        test.append(a)


with open('str.ins', "w", newline='') as f:
    f.write("pif ~" + "\n")  
    writer = csv.writer(f)
    for row in test:
        writer.writerow([row])


### 2. Watertable (modflow_cell_obs.txt)

In [ ]:
import csv
import pandas as pd
import numpy as np

st_date = '1/1/1980'
wd2 = "D:\\Projects\\MiddleBosque\\Analysis\\SWAT+MODFLOW Model_middle_bosque"
df_wt = pd.read_csv(
                    wd2 + "\\modflow_cell_obs.txt",
                    delim_whitespace=True,
                    skiprows=1,
#                     usecols=[1,2],
                    header=None
                    )
df_wt.index = pd.date_range(st_date, periods=len(df_wt))
# print(df_str)

co1L = []
co2D = []



# for i in range(len(df_str)):
#     if ((df_wt.index[i].strftime('%Y%m%d') >= '19850821') & (df_wt.index[i].strftime('%Y%m%d') <= '19860507')):
#         print('true')
df_wt['date'] = df_wt.index.strftime('%Y%m%d')
# df_wt['test'] = CO1L['8/21/1985':'5/7/1986']
# df_wt['test'] = df_wt[(df_wt.index >= '08/21/1985') & (df_wt.index <= '05/07/1986')] = 'w'
df_wt['2nd'] = np.where((df_wt.index >= '08/21/1985') & (df_wt.index <= '05/07/1986'), ('l1 !wt_2nd'+df_wt['date']+'!'), 'l1 w')
df_wt['1st'] = np.where((df_wt.index >= '09/30/1985') & (df_wt.index <= '04/01/1986'), ('!wt_1nd'+df_wt['date']+'!'), ' w')


print(df_wt)

with open('wt.ins', "w", newline='') as f:
    f.write("pif ~" + "\n")
    df_wt.to_csv(f, sep='\t',
                  header=False,
                  index=False,
#                   float_format='%.2f', 
                  line_terminator='\n', 
                  columns=('2nd','1st'),
                  encoding='utf-8')

'''
test = []
for i in range(len(df_str)):
    if df_str.iloc[i, 6] == 'cha53':
        a = 'l1 w w w w w w w w '
        b = '!str_{}{:02d}{:02d}!'.format(df_str.iloc[i, 3], df_str.iloc[i, 1], df_str.iloc[i, 2])
        test.append(a+b)
    else:
        a = 'l1'
        test.append(a)


with open('str.ins', "w", newline='') as f:
    f.write("pif ~" + "\n")  
    writer = csv.writer(f)
    for row in test:
        writer.writerow([row])
'''

In [ ]:
a = np.full((33, 55), 10)

In [ ]:
np.shape(a)

In [ ]:
np.savetxt('test.txtt', a, fmt='%.12e', delimiter='\t')
np.savetxt('vtest.txtt', a/10, fmt='%.12e', delimiter='\t')

In [ ]:
b = np.loadtxt('test.txtt')
b/10

In [ ]:
with open('hk04.dat', 'r') as f:
    data = [x.strip().split() for x in f if x.strip()]
hk = float(data[0][4])
hk

In [ ]:
data_fac = ['hk01.dat', 'hk02.dat', 'hk03.dat', 'sy01.dat', 'sy02.dat', 'sy03.dat']
for i in data_fac:
    if i[:2] == 'hk':
        print('true')



In [ ]:
print(a)

In [ ]:
b/10